In [22]:
using Plots, BenchmarkTools
using LinearAlgebra
# using NAJ

In [30]:
function is_row_wise_diagonally_dominant(A::AbstractMatrix{T}) where T<:Number
    M, N = size(A)[1:2]
    result = true
    for i in 1:N
        D = abs(A[i, i])
        if  sum(abs.(A[i,:])) > 2*D
            result = false
            break
        end
    end
    return result
end

function is_column_wise_diagonally_dominant(A::AbstractMatrix{T}) where T<:Number
    M, N = size(A)[1:2]
    result = true
    for j in 1:M
        D = abs(A[j, j])
        if  sum(abs.(A[:,j])) > 2*D
            result = false
            break
        end
    end
    return result
end

function is_strictly_row_wise_diagonally_dominant(A::AbstractMatrix{T}) where T<:Number
    M, N = size(A)[1:2]
    result = true
    for i in 1:N
        D = abs(A[i, i])
        if  sum(abs.(A[i,:])) ≥ 2*D
            result = false
            break
        end
    end
    return result
end

function is_strictly_column_wise_diagonally_dominant(A::AbstractMatrix{T}) where T<:Number
    M, N = size(A)[1:2]
    result = true
    for j in 1:M
        D = abs(A[j, j])
        if  sum(abs.(A[:,j])) ≥ 2*D
            result = false
            break
        end
    end
    return result
end


function spectral_radius(A::Matrix{T}) where T<:Number
    eigs = eigvals(A)
    return findmax([abs(x) for x in eigs])[1]
end

function optimal_omega(A::Matrix{<:Real})
    D = Diagonal(A)
    U = UpperTriangular(A)
    L = LowerTriangular(A)
    T = - inv(D+L) * U
    return 1/(1+sqrt(1-spectral_radius(T)))
end

optimal_omega (generic function with 1 method)

In [5]:
A = [2 1; 2 -2]
B = [2 4; 1 -3]
println(spectral_radius(A+B), ", ", spectral_radius(A) + spectral_radius(B))
#spectral_radius(A)

6.437171043518958, 6.151051861499603


In [6]:
spectral_radius(B)

3.7015621187164243

In [7]:
eigvals(A)

2-element Vector{Float64}:
 -2.449489742783178
  2.4494897427831788

In [31]:
function check_diagonal_zero(A::Matrix{T}) where T<:Number
    tzero = zero(T)
    if tzro ∈ diag(A) 
        return true
    else
        return false
    end
end



function iteration_jacobi3(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector; 
    etol::Number = 1.0e-10,
    Maxiter::Integer = 100_000)
    @assert size(A)[1] == size(A)[2] == size(b)[1]
    @assert Maxiter > 3
    x = zero(x0)
    
    for niter in 1:Maxiter
        @inbounds for i in 1:length(x0)
            @inbounds for j in 1:length(x0)
                if i ≠ j
                    x[i] += -A[i,j] * x0[j]
                end
            end
            @inbounds x[i] = (x[i]+b[i])/A[i, i] 
        end
        if norm(x .- x0, Inf) / norm(x, Inf) < etol
            break
        else 
            x0 = x[:]
            x = zero(x0)

        end
    end
    return x
end

function iteration_seidel3(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector; 
    etol::Number = 1.0e-10, 
    Maxiter = 100_000)
    @assert size(A)[1] == size(A)[2] == size(b)[1]
    
    x = zero(x0)
    for niter in 1:Maxiter
        @inbounds for i in 1:length(x0)
            @inbounds for j in 1:length(x0)
                if j < i
                    x[i] += - A[i, j]*x[j]
                elseif j>i
                    x[i] += - A[i, j]*x0[j] 
                end
            end
            @inbounds x[i] = (x[i]+b[i])/A[i, i] 
        end
        if norm(x .- x0, Inf) / norm(x, Inf) < etol
            break
        else 
            x0 = x[:]
            x = zero(x0)

        end
    end
    return x
end

function iteration_sor3(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector,
    ω::Real; 
    etol::Number = 1.0e-10, 
    Maxiter = 100_000)

    @assert 0 < ω < 2
    
    x = zero(x0)
    for nitter in 1:Maxiter
        @inbounds for i ∈ 1:length(x0)
            @inbounds for j ∈ 1:length(x0)
                if j < i
                    x[i] += -A[i, j] *x[j]
                elseif j > i
                    x[i] += -A[i, j] *x0[j]
                end
            end
            x[i] = (1-ω)*x0[i] + 1/A[i, i] * (ω*b[i] + ω *  x[i])
        end
        if norm(x .- x0, Inf)/norm(x, Inf)< etol    
            return x
        else 
            x0 = x[:]
            x = zero(x)
        end
    end
    return x
end

function iteration_steepest(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector;
    etol::Number = 1.0e-5, 
    Maxiter = 100_000)

    x = similar(x0)
    for i in 1:Maxiter
        v = b - A*x0
        t = dot(v,(b-A*x0))/dot(v, (A*v))
        x = x0 + t*v
        if norm(A*x-b, Inf)<etol
            nitter = i
            println(nitter)
            return x
        else 
            x0 = x
        end
    end
    return nothing
end


function iteration_orthogonal(
    A::AbstractMatrix, 
    b::Vector, 
    x0::Vector;
    etol::Number = 1.0e-5, 
    Maxiter = 100_000)

    x = similar(x0)
    for i in 1:Maxiter
        v = b - A*x0
        t = dot(v,(b-A*x0))/dot(v, (A*v))
        x = x0 + t*v
        if norm(A*x-b, Inf)<etol
            nitter = i
            println(nitter)
            return x
        else 
            x0 = x
        end
    end
    return nothing
end

iteration_orthogonal (generic function with 1 method)

In [32]:
n=100
A = Tridiagonal(-0.5*ones(n-1), ones(n), -0.5*ones(n-1))
b = [0.5; zeros(n-1)]
x0 = Float64.(collect(1:n))

100-element Vector{Float64}:
   1.0
   2.0
   3.0
   4.0
   5.0
   6.0
   7.0
   8.0
   9.0
  10.0
   ⋮
  92.0
  93.0
  94.0
  95.0
  96.0
  97.0
  98.0
  99.0
 100.0

In [33]:
x1 = iteration_jacobi(A, b, x0, etol=1.0e-10)
x12 = iteration_jacobi3(A, b, x0, etol=1.0e-10)
x2 = iteration_seidel3(A, b, x0, etol=1.0e-10)
x3 = iteration_sor3(A, b, x0, 1.5, etol=1.0e-10)
# x4 = iteration_steepest(A, b, x0, etol = 1.0e-5)

# [norm(b-A*x) for x in [x1, x2, x3, x4]]

UndefVarError: UndefVarError: `iteration_jacobi` not defined

In [34]:
@btime iteration_jacobi3(A, b, x0, etol=1.0e-10)
@btime iteration_seidel3(A, b, x0, etol=1.0e-10)
@btime iteration_sor3(A, b, x0, 1.5, etol=1.0e-10)

  888.334 ms (125615 allocations: 107.34 MiB)
  449.729 ms (62855 allocations: 53.71 MiB)
  173.370 ms (22100 allocations: 18.88 MiB)


100-element Vector{Float64}:
 0.9900990110349311
 0.9801980220654706
 0.970297033090538
 0.9603960441090611
 0.9504950551199771
 0.9405940661222332
 0.9306930771147881
 0.9207920880966138
 0.9108910990666946
 0.9009901100240298
 ⋮
 0.08910891971659195
 0.07920792864717459
 0.06930693757297085
 0.059405946494938765
 0.04950495541403728
 0.039603964331225365
 0.029702973247461018
 0.019801982163700375
 0.009900991080896813

In [35]:
for tt ∈ (0.2, 0.5, 0.8, 1.0, 1.2, 1.5, 1.75, 1.9)
    @btime iteration_sor3(A, b, x0, $tt, etol=1.0e-10)
end


  2.368 s (300002 allocations: 256.35 MiB)
  1.404 s (178224 allocations: 152.29 MiB)
  721.846 ms (92370 allocations: 78.93 MiB)
  491.782 ms (62856 allocations: 53.71 MiB)
  337.135 ms (42756 allocations: 36.53 MiB)
  173.613 ms (22101 allocations: 18.88 MiB)
  76.838 ms (9807 allocations: 8.38 MiB)
  27.540 ms (3537 allocations: 3.02 MiB)


In [36]:
norm(A*x2 - b, Inf)

UndefVarError: UndefVarError: `x2` not defined

In [37]:
x1

UndefVarError: UndefVarError: `x1` not defined

In [38]:
x2

UndefVarError: UndefVarError: `x2` not defined

In [39]:
x3

UndefVarError: UndefVarError: `x3` not defined

In [ ]:
spectral_radius(inv(D)*(L+U))

In [ ]:
spectral_radius(inv(D+L)U)

In [ ]:
x'*x

In [ ]:
dot(x,x)

In [41]:
diagonal(A)

UndefVarError: UndefVarError: `diagonal` not defined

In [43]:
UpperTriangular(A)

4×4 UpperTriangular{Float64, Matrix{Float64}}:
 10.0  -1.0   2.0   0.0
   ⋅   11.0  -1.0   3.0
   ⋅     ⋅   10.0  -1.0
   ⋅     ⋅     ⋅    8.0

In [41]:
A=[32 23;2 0]

2×2 Matrix{Int64}:
 32  23
  2   0

In [27]:
is_strictly_column_wise_diagonally_dominant(A)

false

In [28]:
diag(A)
zero(eltype(A)) ∈ diag(A)

true

In [42]:
spectral_radius(A)

33.378147196982766